In [ ]:
import os
import sys
import importlib
from pathlib import Path

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

# %load_ext autoreload
# %autoreload 2


In [ ]:
# MAIN DIR
main_dir = f'{str(Path(code_dir).parents[0])}/'

# code_dir
code_dir = f'{code_dir}/'
sys.path.append(code_dir)

# scraping dir
scraped_data = f'{code_dir}scraped_data/'

# data dir
data_dir = f'{code_dir}data/'

# df save sir
df_save_dir = f'{data_dir}final dfs/'

# lang models dir
llm_path = f'{data_dir}Language Models'


In [ ]:
import string
import re
import time
import tqdm
import json
import csv
import glob
import pickle
import joblib
import random
import itertools
import unicodedata
import multiprocessing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('ggplot')
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 5000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)
pd.set_option('display.float_format', '{:.3f}'.format)


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.dummy import DummyClassifier
from sklearn.pipeline import FeatureUnion
from sklearn.feature_selection import (SelectFdr, SelectFpr,
                                       SelectFromModel, SelectFwe,
                                       SelectKBest, SelectPercentile, chi2,
                                       f_classif, f_regression,
                                       mutual_info_classif,
                                       mutual_info_regression)
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.linear_model import (LogisticRegression,
                                  PassiveAggressiveClassifier, Perceptron,
                                  SGDClassifier)
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (AdaBoostClassifier, BaggingClassifier,
                              BaggingRegressor, ExtraTreesClassifier,
                              GradientBoostingClassifier,
                              RandomForestClassifier, StackingClassifier,
                              StackingRegressor, VotingClassifier,
                              VotingRegressor)
from sklearn.model_selection import (GridSearchCV, KFold, LeaveOneOut,
                                     RandomizedSearchCV,
                                     RepeatedStratifiedKFold, ShuffleSplit,
                                     StratifiedKFold,
                                     StratifiedShuffleSplit,
                                     cross_val_score, cross_val_predict, cross_validate,
                                     learning_curve, train_test_split)
from sklearn.utils.validation import (check_is_fitted, column_or_1d,
                                      has_fit_parameter)
from sklearn import feature_selection, metrics, set_config, svm, utils
from sklearn.metrics import (ConfusionMatrixDisplay,accuracy_score, balanced_accuracy_score,
                             brier_score_loss, classification_report, cohen_kappa_score,
                             confusion_matrix, f1_score, log_loss,
                             make_scorer, matthews_corrcoef,
                             precision_recall_curve, precision_score,
                             recall_score, roc_auc_score)
from sklearn.calibration import CalibrationDisplay
from sklearn.utils import (check_consistent_length, check_random_state, check_X_y)
from imblearn.pipeline import make_pipeline, Pipeline
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.over_sampling import SMOTE, SMOTENC, RandomOverSampler
from imblearn.under_sampling import (EditedNearestNeighbours, NearMiss,
                                     RandomUnderSampler, TomekLinks)
from xgboost import XGBClassifier

### READ DATA

In [ ]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_for_trainning.pkl').reset_index(drop=True)[:200]


In [ ]:
# Validation split ratios
n_jobs = 1
train_ratio = 0.75
test_ratio = 0.10
validation_ratio = 0.15
test_split = test_size = 1 - train_ratio
validation_split = test_ratio / (test_ratio + validation_ratio)

# Cross-validation
random.seed(42)
np.random.seed(42)
random_state = 42
partition = True
cv = RepeatedStratifiedKFold(
    n_splits=10, n_repeats=3, random_state=random_state)
class_weight = 'balanced'
t = time.time()
cores = multiprocessing.cpu_count()
model_sizes = [300, 100]
scoring = 'recall'
scores = ['recall', 'accuracy', 'f1', 'roc_auc', 'explained_variance', 'matthews_corrcoef']
scorers = {
    'precision_score': make_scorer(precision_score),
    'recall_score': make_scorer(recall_score),
    'accuracy_score': make_scorer(accuracy_score),
}
metrics_list = [
    'Mean Validation Score',
    'Explained Variance',
    'Accuracy',
    'Precision',
    'Recall',
    'F1-score',
    'ROC',
    'AUC',
    'Matthews Correlation Coefficient',
    f'{scoring.title()} Best Threshold',
    f'{scoring.title()} Best Score',
    'Log Loss/Cross Entropy',
    'Classification Report',
    'Confusion Matrix',
]

# Paths
models_save_path = f'{data_dir}classification models/'
table_save_path = f'{data_dir}output tables/'
plot_save_path = f'{data_dir}plots/'

## Vectorizers

In [ ]:
### CountVectorizer
count_ = CountVectorizer()
count_params = {
#     'TfidfVectorizer__stop_words': ['english'],
    'CountVectorizer__analyzer': ['word'],
    'CountVectorizer__ngram_range': [(1, 3)],
    'CountVectorizer__lowercase': [True, False],
    'CountVectorizer__max_df': [0.90, 0.85, 0.80, 0.75, 0.70],
    'CountVectorizer__min_df': [0.10, 0.15, 0.20, 0.25, 0.30],
}
count = [count_, count_params]

### TfidfVectorizer
tfidf_ = TfidfVectorizer()
tfidf_params = {
#     'TfidfVectorizer__stop_words': ['english'],
    'TfidfVectorizer__analyzer': ['word'],
    'TfidfVectorizer__ngram_range': [(1, 3)],
    'TfidfVectorizer__lowercase': [True, False],
    'TfidfVectorizer___use_idf': [True, False],
    'TfidfVectorizer__max_df': [0.90, 0.85, 0.80, 0.75, 0.70],
    'TfidfVectorizer__min_df': [0.10, 0.15, 0.20, 0.25, 0.30],
}
tfidf = [tfidf_, tfidf_params]

## Vectorizers List
vectorizers_list = [count, tfidf]

### BOW FeatureUnion
transformer_list = []
bow_params = {}
for vectorizer_and_params in vectorizers_list:
    transformer_list.append(
        tuple([vectorizer_and_params[0].__class__.__name__, vectorizer_and_params[0]])
    )
    bow_params.update(
        vectorizer_and_params[1]
    )

bow_ = FeatureUnion(
    transformer_list=[transformer_list]
)
bow = [bow_, bow_params]

## Vectorizers List append bow
vectorizers_list.append(bow)

## Vectorizers Dict
vectorizers_pipe = {
    vectorizer_and_params[0].__class__.__name__: vectorizer_and_params
    for vectorizer_and_params in vectorizers_list
}



## Selectors

In [ ]:
### SelectKBest
selectkbest_ = SelectKBest()
selectkbest_params = {
    'SelectKBest__score_func': [f_classif, chi2, mutual_info_classif, f_regression, mutual_info_regression],
    'SelectKBest__k': ['all'],
}
selectkbest = [selectkbest_, selectkbest_params]

### SelectPercentile
selectperc_ = SelectPercentile()
selectperc_params = {
    'SelectPercentile__score_func': [f_classif, chi2, mutual_info_classif, f_regression, mutual_info_regression],
    'SelectPercentile__percentile': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
}
selectperc = [selectperc_, selectperc_params]

### SelectFpr
selectfpr_ = SelectFpr()
selectfpr_params = {
    'SelectFpr__score_func': [f_classif, chi2, mutual_info_classif, f_regression, mutual_info_regression],
}
selectfpr = [selectfpr_, selectfpr_params]

### SelectFdr
selectfdr_ = SelectFdr()
selectfdr_params = {
    'SelectFdr__score_func': [f_classif, chi2, mutual_info_classif, f_regression, mutual_info_regression],
}
selectfdr = [selectfdr_, selectfdr_params]

### SelectFwe
selectfwe_ = SelectFwe()
selectfwe_params = {
    'SelectFwe__score_func': [f_classif, chi2, mutual_info_classif, f_regression, mutual_info_regression],
}
selectfwe = [selectfwe_, selectfwe_params]

## Selectors List
selectors_list = [
    selectkbest, selectperc, selectfpr, selectfdr, selectfwe
]
## Selectors Dict
selectors_pipe = {
    selector_and_params[0].__class__.__name__: selector_and_params
    for selector_and_params in selectors_list
}


## Resamplers

In [ ]:
# Resamplers
### SMOTETomek Resampler
smotetomek_ = SMOTETomek()
smotetomek_params = {
    'SMOTETomek__random_state': [random_state],
    'SMOTETomek__tomek': [TomekLinks(sampling_strategy='majority')],
    'SMOTETomek__n_jobs': [-1],
}
smotetomek = [smotetomek_, smotetomek_params]

### SMOTENC Resampler
smotenc_ = SMOTENC(categorical_features=[[df_manual.columns.get_loc('Warmth'), df_manual.columns.get_loc('Competence')]])
smotenc_params = {
    'SMOTENC__random_state': [random_state],
    'SMOTENC__sampling_strategy': [.6],
}
smotenc = [smotenc_, smotenc_params]

## Selectors List
resamplers_list = [
    # smotetomek,
    smotenc
]

## Selectors Dict
resamplers_pipe = {
    resampler_and_params[0].__class__.__name__: resampler_and_params
    for resampler_and_params in resamplers_list
}

## Classifiers

In [ ]:
# Classifiers
### Dummy Classifier
dummy_ = DummyClassifier()
dummy_params = {
    'DummyClassifier__strategy': [
        'stratified',
        'most_frequent',
        'prior',
        'uniform',
    ],
    'DummyClassifier__random_state': [random_state],
}

dummy = [dummy_, dummy_params]

### Multinomial Naive Bayes
nb_ = MultinomialNB()
nb_params = {
    'MultinomialNB__fit_prior': [True],
    'MultinomialNB__alpha': [0.1, 0.2, 0.3],
}

nb = [nb_, nb_params]

### Bernoulli Naive Bayes
bnb_ = BernoulliNB()
bnb_params = {
    'BernoulliNB__fit_prior': [True],
    'BernoulliNB__alpha': [0.1, 0.2, 0.3],
}

bnb = [bnb_, bnb_params]

### Gaussian Naive Bayes
gnb_ = GaussianNB()
gnb_params = {
    'GaussianNB__var_smoothing': [1e-9],
}

gnb = [gnb_, gnb_params]

### KNeighbors Classifier
knn_ = KNeighborsClassifier()
knn_params = {
    'KNeighborsClassifier__weights': ['uniform'],
    'KNeighborsClassifier__n_neighbors': [2, 5, 15],
    'KNeighborsClassifier__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'KNeighborsClassifier__leaf_size': [30, 50, 100, 200, 300, 500],
    'KNeighborsClassifier__p': [1, 2, 3, 4, 5],
    'KNeighborsClassifier__metric': [
        'minkowski',
        'euclidean',
        'cosine',
        'correlation',
    ],
    'KNeighborsClassifier__metric_params': [None, {'p': 2}, {'p': 3}],
}

knn = [knn_, knn_params]

### Logistic Regression
lr_ = LogisticRegression()
lr_params = {
    'LogisticRegression__penalty': ['l2'],
    'LogisticRegression__random_state': [random_state],
    'LogisticRegression__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'LogisticRegression__max_iter': [100, 200, 300, 500, 1000],
    'LogisticRegression__multi_class': ['ovr', 'multinomial'],
    'LogisticRegression__solver': ['newton-cg', 'liblinear'],
    'LogisticRegression__C': [0.01, 1, 100],
}

lr = [lr_, lr_params]

### Passive Aggressive
pa_ = PassiveAggressiveClassifier()
pa_params = {
    'PassiveAggressiveClassifier__loss': ['hinge', 'squared_hinge'],
    'PassiveAggressiveClassifier__random_state': [random_state],
    'PassiveAggressiveClassifier__fit_intercept': [True, False],
    'PassiveAggressiveClassifier__class_weight': [None, 'balanced'],
    'PassiveAggressiveClassifier__max_iter': [100, 200, 300, 500, 1000],
}

pa = [pa_, pa_params]

### Stochastic Gradient Descent Aggressive
sgd_ = SGDClassifier()
sgd_params = {
    'SGDClassifier__loss': ['hinge', 'squared_hinge'],
    'SGDClassifier__random_state': [random_state],
    'SGDClassifier__fit_intercept': [True, False],
    'SGDClassifier__class_weight': [None, 'balanced'],
    'SGDClassifier__max_iter': [100, 200, 300, 500, 1000],
}

sgd = [sgd_, sgd_params]

### SVM
svm_ = LinearSVC()
svm_params = {
    'LinearSVC__penalty': ['l2'],
    'LinearSVC__loss': ['hinge', 'squared_hinge'],
    'LinearSVC__random_state': [random_state],
    'LinearSVC__max_iter': [100, 200, 300, 500, 1000],
    'LinearSVC__fit_intercept': [True, False],
    'LinearSVC__class_weight': [None, 'balanced'],
    'LinearSVC__multi_class': ['ovr', 'crammer_singer'],
}

svm = [svm_, svm_params]

### Decision Tree
dt_ = DecisionTreeClassifier()
dt_params = {
    'DecisionTreeClassifier__max_depth': [5, 10],
    'DecisionTreeClassifier__criterion': ['gini', 'entropy'],
    'DecisionTreeClassifier__random_state': [random_state],
    'DecisionTreeClassifier__splitter': ['best', 'random'],
    'DecisionTreeClassifier__max_features': [None, 'auto', 'sqrt', 'log2'],
}

dt = [dt_, dt_params]

### Random Forest
rf_ = RandomForestClassifier()
rf_params = {
    'RandomForestClassifier__n_estimators': [10, 20],
    'RandomForestClassifier__n_jobs': [-1],
    'RandomForestClassifier__max_depth': [5, 10],
    'RandomForestClassifier__max_feature': [*np.arange(0.1, 1.1, 0.1)],
    'RandomForestClassifier__random_state': [random_state],
    'RandomForestClassifier__class_weight': [None, 'balanced'],
    'RandomForestClassifier__oob_score': [True],
}

rf = [rf_, rf_params]

### Extra Trees
et_ = ExtraTreesClassifier()
et_params = {
    'ExtraTreesClassifier__n_estimators': [10, 20],
    'ExtraTreesClassifier__n_jobs': [-1],
    'ExtraTreesClassifier__max_depth': [5, 10],
    'ExtraTreesClassifier__max_feature': [*np.arange(0.1, 1.1, 0.1)],
    'ExtraTreesClassifier__random_state': [random_state],
    'ExtraTreesClassifier__criterion': ['gini', 'entropy', 'log_loss'],
    'ExtraTreesClassifier__class_weight': [None, 'balanced'],
}

et = [et_, et_params]

### Gradient Boosting
gbc_ = GradientBoostingClassifier()
gbc_params = {
    'GradientBoostingClassifier__max_depth': [5, 10],
    'GradientBoostingClassifier__criterion': ['gini', 'entropy'],
    'GradientBoostingClassifier__random_state': [random_state],
    'GradientBoostingClassifier__n_estimators': [10, 20],
    'GradientBoostingClassifier__loss': ['deviance', 'exponential'],
    'GradientBoostingClassifier__subsample': [*np.arange(0.1, 1.1, 0.1)],
    'GradientBoostingClassifier__max_features': [None, 'auto', 'sqrt', 'log2'],
}

gbc = [gbc_, gbc_params]

### AdaBoost
ada_ = AdaBoostClassifier()
ada_params = {
    'AdaBoostClassifier__max_depth': [5, 10],
    'AdaBoostClassifier__criterion': ['gini', 'entropy'],
    'AdaBoostClassifier__random_state': [random_state],
    'AdaBoostClassifier__n_estimators': [50, 100, 150],
    'AdaBoostClassifier__base_estimator': [
        SVC(probability=True, kernel='linear'),
        LogisticRegression(),
        MultinomialNB(),
    ],
}

ada = [ada_, ada_params]

### XGBoost
xgb_ = XGBClassifier()
xgb_params = {
    'XGBClassifier__max_depth': [5, 10],
    'XGBClassifier__learning_rate': [0.05],
    'XGBClassifier__n_estimators': [1000],
    'XGBClassifier__seed': [42],
    'XGBClassifier__nthread': [1, 2, 3, 4],
    'XGBClassifier__objective': ['binary:logitraw', 'binary:logistic', 'binary:hinge'],
    'XGBClassifier__eval_metric': ['auc', 'rmse', 'rmsle', 'logloss'],
    'XGBClassifier__sample_type': ['weighted', 'uniform'],
}

xgb = [xgb_, xgb_params]

### MLP Classifier
mlpc_ = MLPClassifier()
mlpc_params = {
    'MLPClassifier__hidden_layer_sizes': [(100,), (50,), (25,), (10,), (5,), (1,)],
    'MLPClassifier__activation': ['identity', 'logistic', 'tanh', 'relu'],
    'MLPClassifier__solver': ['lbfgs', 'sgd', 'adam'],
    'MLPClassifier__learning_rate': ['constant', 'invscaling', 'adaptive'],
    'MLPClassifier__random_state': [random_state],
}

mlpc = [mlpc_, mlpc_params]

### MLP Regressor
mlpr_ = MLPRegressor()
mlpr_params = {
    'MLPRegressor__hidden_layer_sizes': [(100,), (50,), (25,), (10,), (5,), (1,)],
    'MLPRegressor__activation': ['identity', 'logistic', 'tanh', 'relu'],
    'MLPRegressor__solver': ['lbfgs', 'sgd', 'adam'],
    'MLPRegressor__learning_rate': ['constant', 'invscaling', 'adaptive'],
    'MLPRegressor__random_state': [random_state],
}

mlpr = [mlpr_, mlpr_params]

# ## Stacking and Voting Classifiers
# estimators = [
#     ('Multinomial Naive Bayes', MultinomialNB()),
#     (
#         'Logistic Regression',
#         LogisticRegression(random_state=42, class_weight='balanced'),
#     ),
# ]

# ### Voting Classifier
# voting = VotingClassifier(estimators=estimators)
# voting_params = {
#     'VotingClassifier__estimators': [
#         ('dummy', dummy, params_dummy_freq),
#         ('dummy', dummy, params_dummy_stratified),
#         ('dummy', dummy, params_dummy_uniform),
#         ('nb', nb, params_nb),
#         ('bnb', bnb, params_bnb),
#         ('gnb', gnb, params_gnb),
#         ('knn', knn, params_knn),
#         ('lr', lr, params_lr),
#         ('pa', pa, params_pa),
#         ('sgd', sgd, params_sgd),
#         ('svm', svm, params_svm),
#         ('dt', dt, params_dt),
#         ('rf', rf, params_rf),
#         ('gbc', gbc, params_gbc),
#         ('ada', ada, params_ada),
#         ('xgb', xgb, params_xgb),
#         ('mlpc', mlpc, params_mlpc),
#         ('mlpr', mlpr, params_mlpr),
#     ],
#     'VotingClassifier__voting': ['hard', 'soft'],
#     'VotingClassifier__weights': [None, [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]],
# }

# ### Stacking Classifier
# stacking = StackingClassifier(estimators=estimators)
# # final_estimator = LogisticRegression(random_state=random_state, class_weight=class_weight)
# final_estimator = RandomForestClassifier(
#     random_state=42, class_weight={0: 1, 1: 2}
# )
# stacking_params = {
#     'StackingClassifier__estimator': [
#         ('dummy', dummy),
#         ('nb', nb),
#         ('bnb', bnb),
#         ('gnb', gnb),
#         ('knn', knn),
#         ('lr', lr),
#         ('pa', pa),
#         ('sgd', sgd),
#         ('svm', svm),
#         ('dt', dt),
#         ('rf', rf),
#         ('gbc', gbc),
#         ('ada', ada),
#         ('mlpc', mlpc),
#     ],
#     'StackingClassifier__cv': [3, 5, 7, 9, 11, 13, 15],
#     'StackingClassifier__n_jobs': [-1],
#     'StackingClassifier__stack_method': ['auto', 'predict_proba', 'decision_function', 'predict'],
#     'StackingClassifier__passthrough': [True, False],
# }

## Classifiers Pipe list
classifers_list = [
    dummy, nb,
#     bnb, gnb, knn, lr, pa, sgd, svm, dt, rf, gbc, ada, xgb, mlpc, mlpr
]

## Classifiers Pipe dict
classifiers_pipe = {
    classifier_and_params[0].__class__.__name__: classifier_and_params
    for classifier_and_params in classifers_list
}

## Stacking and Voting Classifiers
voting_stacking_params = {}
for classifier_and_params in classifers_list:
    voting_stacking_estimators = tuple([classifier_and_params[0].__class__.__name__, classifier_and_params[0]])
    voting_stacking_params.update(classifier_and_params[1])

### Voting Classifier
voting_ = VotingClassifier(estimators = voting_stacking_estimators)
voting_params = {
#     'VotingClassifier__estimators': estimators,
    'VotingClassifier__voting': ['soft', 'hard'],
    'VotingClassifier__weights': [None],
}

voting = [voting_, voting_params]

### Stacking Classifier
stacking_ = StackingClassifier(estimators = voting_stacking_estimators)
stacking_params = {
#     'StackingClassifier__estimator': estimators,
    'StackingClassifier__cv': cv,
    'StackingClassifier__n_jobs': [-1],
    'StackingClassifier__stack_method': ['auto', 'predict_proba', 'decision_function', 'predict'],
    'StackingClassifier__passthrough': [True, False],
}

stacking = [stacking_, stacking_params]

# Add stacking and voting classifiers to classifiers pipe dict
voting_stacking_params.update(voting[1])
classifiers_pipe[voting[0].__class__.__name__] = voting_stacking_params

# voting_stacking_params.update(stacking[1])
# classifiers_pipe[stacking[0].__class__.__name__] = voting_stacking_params


In [ ]:
def make_table_df(vectorizers_pipe, selectors_pipe, classifiers_pipe, analysis_columns, metrics_list):

    # Table df
    index = pd.MultiIndex.from_product(
        [list(map(lambda classifier: classifier, classifiers_pipe.keys()))],
        names=['Classifiers'],
    )
    columns = pd.MultiIndex.from_product(
        [
            analysis_columns,
            list(map(lambda vectorizer: vectorizer, vectorizers_pipe.keys())),
            list(map(lambda selector: selector, selectors_pipe.keys())),
            metrics_list,
        ],
        names=['Variable', 'Vectorizer', 'Selector', 'Measures'],
    )
    table_df = pd.DataFrame(index=index, columns=columns)

    return table_df


In [ ]:
def split_data(df_manual, col, text_col, analysis_columns):

    train_ratio = 0.75
    test_ratio = 0.10
    validation_ratio = 0.15
    test_split = 1 - train_ratio
    validation_split = test_ratio / (test_ratio + validation_ratio)

    # BOW Split
    print(f'Splitting data into training and testing sets:\nRatios: test size = {test_split}, train_size = {1-test_split}')
    df_manual.dropna(subset=analysis_columns, how='any', inplace=True)

    train, test = train_test_split(
        df_manual, test_size=test_split, train_size = 1-test_split, random_state=random_state
    )

    X_train = np.array([x for x in train[f'{str(text_col)}'].astype('str').values])
    y_train = column_or_1d(train[str(col)].astype('int64').values, warn=True)

    X_test = np.array([x for x in test[f'{str(text_col)}'].astype('str').values])
    y_test = column_or_1d(test[str(col)].astype('int64').values, warn=True)

    print(f'Done splitting data into training and testing sets.')
    print('-'*10)
    print(f'Training set shape: {y_train.shape}')
    print(f'Testing set shape: {y_test.shape}')
    print('-'*10)

    return train, X_train, y_train, test, X_test, y_test


In [ ]:
def plot_confusion_matrix_percentage(
    cm, col, vectorizer_name, selector_name, classifier_name
):
    plt.title(
        f'Confusion Matrix Heatmap Plot {str(col)} - {vectorizer_name} + {selector_name} + {classifier_name}',
        fontsize=14.0,
    )
    group_names = ['True Neg', 'False Pos', 'False Neg', 'True Pos']
    group_counts = [f'{value:0.0f}\n' for value in cm.flatten()]
    group_percentages = [f'{value:.2%}' for value in cm.flatten() / np.sum(cm)]
    labels = [f'{v1}\n{v2}' for v1, v2 in zip(group_counts, group_percentages)]
    labels = np.asarray(labels).reshape(cm.shape[0], cm.shape[1])
    cm_heatmap = sns.heatmap(
        cm, cmap='PuBu', annot=labels, fmt='', annot_kws={'size': 12.0}
    )
    plt.xlabel('Predicted', fontsize=12.0)
    plt.ylabel('Actual', fontsize=12.0)
    plt.tight_layout()
    plt.show()
    
    for image_save_format in ['eps', 'png']:
        cm_heatmap.figure.savefig(
            plot_save_path
            + f'Confusion Matrix Heatmap {str(col)} - {vectorizer_name} + {selector_name} + {classifier_name}.{image_save_format}',
            format=image_save_format,
            dpi=3000, bbox_inches='tight'
        )
        
    plt.clf()
    plt.cla()
    plt.close()

    return cm_heatmap


In [ ]:
def plot_metrics(cm, cm_curve, roc_curve, pr_curve, calibration_curve, recall, precision, no_skill, y_validate):
    # Plots
    print('=' * 20)
    print('Plotting:')
    ## Confusion Matrix
#     plt.clf()
#     plt.cla()
#     plt.close()
    print('-' * 20)
    print(f'Confusion Matrix:')
    plt.title(
        f'Confusion Matrix {str(col)} - {vectorizer_name} + {selector_name} + {classifier_name}',
        fontsize=16,
    )
    cm_curve.plot(cmap=plt.cm.Blues)
#     plt.clf()
#     plt.cla()
#     plt.close()

    ## Confusion Matrix Heatmap
    print('-' * 20)
    print(f'Confusion Matrix Heatmap Plot:')
    cm_heatmap = plot_confusion_matrix_percentage(cm, col, vectorizer_name, selector_name, classifier_name)
    
    ## ROC Curve
    print('-' * 20)
    print(f'ROC Curve:')      
#     plt.title(
#         f'ROC Curve {str(col)} - {vectorizer_name} + {selector_name} + {classifier_name}', fontsize=16,
#     )
    roc_curve.plot()
#     plt.clf()
#     plt.cla()
#     plt.close()

    ## PR Curve
    print('-' * 20)
    print(f'Precision Recall Curve:')
#     plt.title(
#         f'Precision-Recall Curve {str(col)} - {vectorizer_name} + {selector_name} + {classifier_name}', fontsize=16,
#     )
#     plt.ylabel('Precision', fontsize=12.0)
#     plt.xlabel('Recall', fontsize=12.0)
    pr_curve.plot()
#     plt.clf()
#     plt.cla()
#     plt.close()
    
    ## Calibration Curve
    print('-' * 20)
    print(f'Calibration Curve:')
#     plt.title(
#         f'Calibration Curve {str(col)} - {vectorizer_name} + {selector_name} + {classifier_name}', fontsize=16,
#     )
    calibration_curve.plot()
#     plt.clf()
#     plt.cla()
#     plt.close()
    print('=' * 20)

    # Save Plots
    print('Saving plots.')
    for image_save_format in ['eps', 'png']:
        cm_curve.figure_.savefig(
            plot_save_path
            + f'Confusion Matrix {str(col)} - {vectorizer_name} + {selector_name} + {classifier_name}.{image_save_format}',
            format=image_save_format,
            dpi=3000, bbox_inches='tight'
        )

#         cm_heatmap.figure.savefig(
#             plot_save_path
#             + f'Confusion Matrix Heatmap {str(col)} - {vectorizer_name} + {selector_name} + {classifier_name}.{image_save_format}',
#             format=image_save_format,
#             dpi=3000, bbox_inches='tight'
#         )

        roc_curve.figure_.savefig(
            plot_save_path
            + f'ROC Curve {str(col)} - {vectorizer_name} + {selector_name} + {classifier_name}.{image_save_format}',
            format=image_save_format,
            dpi=3000, bbox_inches='tight'
        )

        pr_curve.figure_.savefig(
            plot_save_path
            + f'Precision Recall Curve {str(col)} - {vectorizer_name} + {selector_name} + {classifier_name}.{image_save_format}',
            format=image_save_format,
            dpi=3000, bbox_inches='tight'
        )

        calibration_curve.figure_.savefig(
            plot_save_path
            + f'Calibration Curve {str(col)} - {vectorizer_name} + {selector_name} + {classifier_name}.{image_save_format}',
            format=image_save_format,
            dpi=3000, bbox_inches='tight'
        )



In [ ]:
# Save Model
def saving_model_and_table(col, table_df, estimator, vectorizer_name, selector_name, classifier_name):

    # Save classifier
    print(f'Saving Model and Table for {vectorizer_name} + {classifier_name}.')
    table_df.to_csv(table_save_path + 'Classifiers Table.csv')
    table_df.to_pickle(table_save_path + 'Classifiers Table.pkl')
    table_df.to_excel(table_save_path + 'Classifiers Table.xlsx')
    table_df.to_latex(table_save_path + 'Classifiers Table.tex')
    table_df.to_markdown(table_save_path + 'Classifiers Table.md')

    with open(f'{models_save_path}Estimator {str(col)} - {vectorizer_name} + {selector_name} + {classifier_name}.pkl', 'wb') as f:
        joblib.dump(estimator, f)
#     with open(f'{models_save_path}Vectorizer {str(col)} - {vectorizer_name} + {classifier_name}.pkl', 'wb') as f:
#         joblib.dump(vectorizer, f)
#     with open(f'{models_save_path}Selector {str(col)} - {vectorizer_name} + {classifier_name}.pkl', 'wb') as f:
#         joblib.dump(selector, f)


In [ ]:
def evaluation(estimator, col, vectorizer_name, selector_name, classifier_name, X_test, y_test, y_test_pred, y_test_pred_prob, best_score, scoring):

    cross_validate_score_noscoring = cross_validate(
        estimator,
        X_test,
        y_test,
        cv=cv,
        return_train_score=True,
    )

    cross_validate_score = cross_validate(
        estimator,
        X_test,
        y_test,
        cv=cv,
        return_train_score=True,
        scoring=scores,
    )

    no_skill = len(y_test[y_test == 1]) / len(y_test)
    mean_validation_score = cross_validate_score_noscoring.get('test_score').mean()
    explained_variance = cross_validate_score.get('test_explained_variance').mean()
    accuracy = metrics.accuracy_score(y_test, y_test_pred)
    precision = metrics.precision_score(y_test, y_test_pred, pos_label=1, labels=[1, 0])
    recall = metrics.recall_score(y_test, y_test_pred, pos_label=1, labels=[1, 0])
    f1 = metrics.f1_score(y_test, y_test_pred)
    mcc = metrics.matthews_corrcoef(y_test, y_test_pred)
    loss = log_loss(y_test, y_test_pred_prob)
    report = classification_report(y_test, y_test_pred)
    cm = metrics.confusion_matrix(y_test, y_test_pred)
    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    roc_auc = metrics.roc_auc_score(y_test, y_test_pred)
    fpr, tpr, threshold = metrics.roc_curve(y_test, y_test_pred_prob, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    precision_pr, recall_pr, threshold_pr = metrics.precision_recall_curve(y_test, y_test_pred_prob, pos_label=1)
    cm_curve = metrics.ConfusionMatrixDisplay.from_predictions(
        y_test, y_test_pred,
    )
    roc_curve = metrics.RocCurveDisplay.from_predictions(
        y_test, y_test_pred, pos_label=1, name=f'{str(col)} - {vectorizer_name} + {selector_name} + {classifier_name}'
    )
    pr_curve = metrics.PrecisionRecallDisplay.from_predictions(
        y_test, y_test_pred, pos_label=1, name=f'{str(col)} - {vectorizer_name} + {selector_name} + {classifier_name}'
    )
    calibration_curve = CalibrationDisplay.from_predictions(
        y_test, y_test_pred, pos_label=1, name=f'{str(col)} - {vectorizer_name} + {selector_name} + {classifier_name}'
    )

    table_df.loc[
        (classifier_name), (col, vectorizer_name, selector_name, 'Mean Validation Score')
    ] = float(mean_validation_score)
    table_df.loc[
        (classifier_name), (col, vectorizer_name, selector_name, 'Explained Variance')
    ] = float(explained_variance)
    table_df.loc[
        (classifier_name), (col, vectorizer_name, selector_name, 'Accuracy')
    ] = float(accuracy)
    table_df.loc[
        (classifier_name), (col, vectorizer_name, selector_name, 'Precision')
    ] = float(precision)
    table_df.loc[
        (classifier_name), (col, vectorizer_name, selector_name, 'Recall')
    ] = float(recall)
    table_df.loc[
        (classifier_name), (col, vectorizer_name, selector_name, 'F1-score')
    ] = float(f1)
    table_df.loc[
        (classifier_name), (col, vectorizer_name, selector_name, 'Matthews Correlation Coefficient'),
    ] = float(mcc)
#     table_df[
#         (classifier_name), (col, vectorizer_name, selector_name, f'{scoring.title()} Best Threshold')
#     ] = table_df[
#         (classifier_name), (col, vectorizer_name, selector_name, f'{scoring.title()} Best Threshold')
#     ].astype(object)
    table_df.loc[
        (classifier_name), (col, vectorizer_name, selector_name, f'{scoring.title()} Best Threshold'),
    ] = str(threshold)
    table_df.loc[
        (classifier_name), (col, vectorizer_name, selector_name, f'{scoring.title()} Best Score'),
    ] = float(best_score)
    table_df.loc[
        (classifier_name), (col, vectorizer_name, selector_name, 'Log Loss/Cross Entropy'),
    ] = float(loss)
    table_df.loc[
        (classifier_name), (col, vectorizer_name, selector_name, 'Classification Report')
    ] = report
#     table_df[
#         (classifier_name), (col, vectorizer_name, selector_name, f'{scoring.title()} Confusion Matrix')
#     ] = table_df[
#         (classifier_name), (col, vectorizer_name, selector_name, f'{scoring.title()} Confusion Matrix')
#     ].astype(object)
    table_df.loc[
        (classifier_name), (col, vectorizer_name, selector_name, 'Confusion Matrix')
    ] = str(cm)
#     table_df[
#         (classifier_name), (col, vectorizer_name, selector_name, f'{scoring.title()} Normalized Confusion Matrix')
#     ] = table_df[
#         (classifier_name), (col, vectorizer_name, f'{scoring.title()} Normalized Confusion Matrix')
#     ].astype(object)
    table_df.loc[
        (classifier_name), (col, vectorizer_name, selector_name, 'Normalized Confusion Matrix')
    ] = str(cm_normalized)
    table_df.loc[
        (classifier_name), (col, vectorizer_name, selector_name, 'ROC')
    ] = float(roc_auc)
    table_df.loc[
        (classifier_name), (col, vectorizer_name, selector_name, 'AUC')
    ] = float(auc)

    print('=' * 20)
    print('~' * 20)
    print(f' Metrics for {str(col)} - {vectorizer_name} + {selector_name} + {classifier_name}')
    print('~' * 20)
    print(f'Classification Report:\n {report}')
    print('-' * 20)
    print(f'Mean Validation Score: {mean_validation_score}')
    print('-' * 20)
    print(f'Recall score: {recall}')
    print('-' * 20)
    print(f'Accuracy score: {accuracy}')
    print('-' * 20)
    print(f'Precision score: {precision}')
    print('-' * 20)
    print(f'F1 score: {f1}')
    print('-' * 20)
    print(f'Matthews correlation coefficient: {mcc}')
    print('-' * 20)
    print(f'{scores[0].title()} best score: {best_score}')
    print('-' * 20)
    print(f'{scores[0].title()} best threshold: {threshold}')
    print('-' * 20)
    print(f'Confusion Matrix:\n', cm)
    print('-' * 20)
    print(f'Confusion Matrix Normalized:\n', cm_normalized)
    print('-' * 20)
    print('=' * 20)
    
    # Plot Metrics
    plot_metrics(cm, cm_curve, roc_curve, pr_curve, calibration_curve, recall, precision, no_skill, y_test)

    return table_df, mean_validation_score, explained_variance, accuracy, precision, recall, f1, mcc, loss, report, cm, cm_normalized


In [ ]:
df_manual = pd.read_pickle(f'{df_save_dir}df_manual_for_trainning.pkl').reset_index(drop=True)[:200]


In [ ]:
analysis_columns = ['Warmth', 'Competence']
text_col = 'Job Description spacy_sentencized'

table_df = make_table_df(vectorizers_pipe, selectors_pipe, classifiers_pipe, analysis_columns, metrics_list)

for col in analysis_columns:

    print('-'*20)
    print(f'{"="*30} TRAINING {col.upper()} {"="*30}')
    print('-'*20)

    if (len(df_manual[df_manual[str(col)].map(df_manual[str(col)].value_counts() > 1)]) != 0):

        # BOW Split
        train, X_train, y_train, test, X_test, y_test = split_data(df_manual, col, text_col, analysis_columns)

        for (vectorizer_name, vectorizer_and_params), (selector_name, selector_and_params), (resampler_name, resampler_and_params), (classifier_name, classifier_and_params) in itertools.product(vectorizers_pipe.items(), selectors_pipe.items(), resamplers_pipe.items(), classifiers_pipe.items()):

            vectorizer = vectorizer_and_params[0]
            vectorizer_params = vectorizer_and_params[1]

            selector = selector_and_params[0]
            selector_params = selector_and_params[1]

            classifier = classifier_and_params[0]
            classifier_params = classifier_and_params[1]

            # Pipeline
            ## Steps
            if col == 'Warmth':
                steps = [
                    (vectorizer_name, vectorizer),
                    ('SMOTENC', SMOTENC(
                        categorical_features = [
                            df_manual.columns.get_loc('Warmth'),
                            df_manual.columns.get_loc('Competence')
                        ], random_state=random_state, sampling_strategy=.6
                    )),
                    # (resampler_name, resampler),
                    (classifier_name, classifier)
                ]
            else:
                steps = [
                    (vectorizer_name, vectorizer),
                    (selector_name, selector),
                    (classifier_name, classifier)
                ]

            ## Params
            param_grid = {
                **vectorizer_params,
                **selector_params,
                **classifier_params,
            }

            ## Pipeline
            pipe = Pipeline(steps=steps)

            # Search
            print('-'*20)
            print(f'{"="*30} Using GridSearchCV {"="*30}')
            print('-'*20)
            print(f'GridSearchCV with:\nPipe:\n{pipe}\nParams:\n{param_grid}')
            print('+'*30)
            search = GridSearchCV(
                estimator=pipe,
                param_grid=param_grid,
                n_jobs=-1,
                scoring=scores,
                cv=cv,
                refit=scores[0],
                return_train_score=True,
            )

            # Fit SearchCV
            searchcv = search.fit(X_train, y_train)

            # Best Parameters
            best_index = searchcv.best_index_
            cv_results = sorted(searchcv.cv_results_)
            best_params = searchcv.best_params_
            best_score = searchcv.best_score_
            n_splits = searchcv.n_splits_
            estimator = searchcv.best_estimator_
            y_train_pred = estimator.predict(X_train)

            # Identify and name steps in estimator
            vectorizer = estimator[0]
            vectorizer_name = vectorizer.__class__.__name__
            selector = estimator[1]
            selector_name = selector.__class__.__name__

            if col == 'Warmth':
                resampler = estimator[2]
                resampler_name = resampler.__class__.__name__
                classifier = estimator[3]
                classifier_name = classifier.__class__.__name__
            else:
                classifier = estimator[2]
                classifier_name = classifier.__class__.__name__

            print('=' * 20)
            print(f'Best index for {scores[0]}: {best_index}')
            print(f'Best estimator for {scores[0]}: {estimator}')
            print(f'Best params for {scores[0]}: {best_params}')
            print(f'Best score for {scores[0]}: {best_score}')
            print(f'Number of splits for {scores[0]}: {n_splits}')

            print('-' * 20)
            train_report = classification_report(y_train, y_train_pred)
            print(f'Training Classification Report:\n{train_report}')
            print(f'Training Confusion Matrix:')
            plt.title(
                f'Training Confusion Matrix {str(col)} - {vectorizer_name} + {selector_name} + {classifier_name}', fontsize=16
            )
            train_cm = metrics.ConfusionMatrixDisplay.from_estimator(
                estimator, X_train, y_train
            ).plot(xticks_rotation='vertical', cmap=plt.cm.Blues)
            plt.clf()
            plt.cla()
            plt.close()
            print('=' * 20)

            # Make the predictions
            if hasattr(searchcv, 'predict_proba'):
                searchcv_predict_attr = searchcv.predict_proba
            elif hasattr(searchcv, '_predict_proba_lr'):
                searchcv_predict_attr = searchcv._predict_proba_lr
            score = searchcv.score(X_test, y_test)
            y_test_pred = searchcv.predict(X_test)
            y_test_pred_prob = searchcv_predict_attr(X_test)[:, 1]
#             y_validate_pred = searchcv.predict(X_validate)
#             y_validate_pred_prob = searchcv_predict_attr(X_validate)[:, 1]
#             y_validate_pred = y_validate_pred_prob

            # Fit Best Model
            print(f'Fitting {estimator}.')
            estimator.set_params(**estimator.get_params())
            estimator = estimator.fit(X_train, y_train)

            # Evaluate Model
            table_df, mean_validation_score, explained_variance, accuracy, precision, recall, f1, mcc, loss, report, cm, cm_normalized = evaluation(
                estimator, col, vectorizer_name, selector_name, classifier_name, X_test, y_test, y_test_pred, y_test_pred_prob, best_score, scoring
            )

            # Save Vectorizer, Selector, and Classifier
            saving_model_and_table(col, table_df, estimator, vectorizer_name, selector_name, classifier_name)

print('#'*40)
print('DONE!')
print('#'*40)


In [ ]:
print('yes')